Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and student ID below:

In [ ]:
NAME1 = ""
NAME2 = ""
NAME3 = ""
ID1 = "" ## Your student id
ID2 = ""
ID3 = ""

---

# Lab 4 Distance Measuring Using FMCW signals 

In the previous labs, you have generated a FMCW wave, i.e.
$$
    S(t)=\cos \left(2 \pi\left(f_{\min }+\frac{B}{2 T}\right) t\right)
$$

Due to propogation delay, the received signal would be
$$
R(t) = A \cos \left(2 \pi\left(f_{\min }+\frac{B}{2 T}\left(t-t_{d}\right)\right)\left(t-t_{d}\right)\right)
$$
where $A$ denotes the attenuation factor while $t_d$ represents the time delay required for the signal to travel from the transmitter to the receiver.

Then We can multiply $S(t)$ and $R(t)$ and filter the high-frequency part. Suppose the distance between recordable device and sender is $d_r$, then we have $t_d =d_r/ v_s$. In other word,

$$
    V(t)=\frac{A}{2} \cos \left(2 \pi f_{\min } \frac{d_r}{v_s}+\left(\frac{2 \pi B d_r t}{v_s T}-\frac{\pi B d_r^{2}}{v_s^{2} T}\right)\right)
$$

Therefore, we can observe a peek at $f = Bd_r/v_sT$ in the frequency domain.

We have prepared one data for you to get a hand-on perception. You can load the data by

In [ ]:
import numpy as np
import sounddevice as sd
# You may need run `conda install -c conda-forge libsndfile` if you use Mac M1
import soundfile as sf
from scipy import signal
from scipy.signal import butter, filtfilt
from matplotlib import pyplot as plt

In [ ]:
fmcw_receive,fs = sf.read('./data/fmcw_receive_data.wav')
# sd.play(fmcw_receive,fs)
# fmcw_receive.reshape(-1,1)

The parameter of sent FMCW wave is

In [ ]:
fs = int(48e3)
f0 = 18000
f1 = 20500
T = 0.04
iteration = 88
zero = True
zero_time = 0.04

And you can read the sent FMCW wave

In [ ]:
fmcw, _ = sf.read('./data/fmcw.wav',dtype='float32')

<b><font color="red" size=5>Checkpoints (10 points)</font></b>

Implement `fft_raw()` function. You have to perform FFT on `fmcw_receive` and `fmcw`. You are required to plot their FFT curves. 

In [ ]:
def fft_raw():
    '''
        Perform FFT on fmcw and fmcw_receive and plot their curves
        Pay attention to the valid information, the frequency range and the axis name
        You can use zero padding to enhance the FFT resolution.(optional)
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    

In [ ]:
fft_raw()

Next, we will preprocess the received signals. We will guide you throughout the process. Observing the spectrum above, we first need to filter the background noise. 



In [ ]:
def filt(s, filt_type, 
           fs, order, 
           f0, 
           f1 = None):
    '''
        s         :   input signal
        filt_type :   ["bandpass","highpass","lowpass"]
        fs        :   sampling rate
        order     :   Order of batterworth filter
        f0,f1     :   Critical frequency or frequencies
                      For bandpass, f0 and f1 need to be specified (f0 < f1)
                      For highpass or lowpass, f1 can be ignored
                      For a Butterworth filter, this is the point at which the gain drops to 
                      1/sqrt(2) that of the passband (the “-3 dB point”)
        
        Return    :   filted 
                     
    '''
    if filt_type == "bandpass":
        assert(f0<f1)
        b, a = signal.butter(order, (f0/(fs/2),f1/(fs/2)), filt_type) 
    else:
        b, a = signal.butter(order, f0/(fs/2), filt_type) 
    filted = signal.filtfilt(b, a, s)  
    return filted

Afterwards, the received signal has to be aligned with the the input signals. We will finish this in `preprocess()` function.

<b><font color="red" size=5>Checkpoints (15 points)</font></b>

Implement `preprocess()` function, where you are required to 

- Fill in the blanks
- Find the starting point of the received signals
- Align the received and sent signals

In [ ]:
def preprocess():
    # Step 1
    # Filter the received signal against the background noise using `filt()` defined above.
    # You need to specify these parameters based on your observation.
    # You should store the filtered signal in `fmcw_receive_filt`
    
    fmcw_receive_filt = np.array([])
    
    #####################################################
    # YOUR CODE HERE
    raise NotImplementedError()
    #####################################################
    
    assert(fmcw_receive_filt.shape[0] == fmcw_receive.shape[0])
    
    # Step 2
    # Find the starting point
    # It may be tricky. You can look at the time-domain curve and find the first index of reflected chirp
    # It is a rough estimate. So the outcome may not be that accurate.
    # You can find some algorithms (e.g. cross-correlation) to improve the accuracy.
    # Store your output in `start`.
    
    start = None
    #####################################################
    # YOUR CODE HERE
    raise NotImplementedError()
    #####################################################
    
    assert(start != None)
    
    # Step 3
    # You need to make sure the received and sent signals have the same shape
    # For data between [0,start], you have to fill the sent data with 0
    # You need to padding zero to whichever is shorter to align
    # The transformed sent data should be stored in fmcw_p
    
    fmcw_p = np.array([])
    
    #####################################################
    # YOUR CODE HERE
    raise NotImplementedError()
    #####################################################
    
    assert(fmcw_p.shape[0]==fmcw_receive_filt.shape[0])
    print(fmcw_receive_filt.shape[0])
    
    return fmcw_p, fmcw_receive_filt, start
    
    
    

In [ ]:
fmcw_p, fmcw_receive_filt, start = preprocess()

By now, you have the sent signals and received signals. They are aligned to be multiplied. Here we can come to the distance measuring part. As aforementioned, we first need to multiply the two signals. Then we need to perform FFT and find the peaks. At last, we compute the distance from the peak index. 

<b><font color="red" size=5>Checkpoints (15 points)</font></b>

Implement `compute_distance()`. You are required to follow the instructions and fill the blanks.

In [ ]:
def compute_distance(fmcw_p = fmcw_p,
                     fmcw_receive_filt = fmcw_receive_filt,
                     start = start):
    
    # Step 1:
    # Multiply fmcw_p and fmcw_receive_filt
    # The output should be the same as input
    # Store output in fmcw_mul
    
    fmcw_mul = np.array([])
    
    #####################################################
    # YOUR CODE HERE
    raise NotImplementedError()
    #####################################################
    
    assert(fmcw_mul.shape[0] == fmcw_p.shape[0])
    
    # Step 2:
    # Performing FFT to each chirp and find the peak index
    # Note that you need to perform FFT in each period of chirp,
    # so that you can get the frequency change continuously
    # You are stongly encouraged to visualize the fft and see the peaks
    # You may use `start` in your implemention
    # You can various peak detection algorithms online
    # Store your peak index in `idxs`:list
    
    idxs = []
    
    ####################################################
    # YOUR CODE HERE
    raise NotImplementedError()
    #####################################################
    assert(len(idxs) > 0)
    
    # Step 3
    # When you get the peak index, you have to convert them to real freq
    # And you can use the formula stated above to compute the distance
    
    out = [] 
    B = f1 - f0
    v_s = 340 # speed of sound
    for idx in idxs:
        
        d = 0
        
        # Computing the distance of each idx represents
        
        #####################################################
        # YOUR CODE HERE
        raise NotImplementedError()
        #####################################################
        
        assert(d>0)
        
        out.append(d)
        # print(d)
    distance = np.mean(out)
    return distance
    

You can use `compute_distance()` to see the results. If you see the values, it means you have  compute the distance using acoustic signals. 

By now, you have implemented all the primary tasks. Congratulations! 

If you are still interested in this topic, you are welcome to implement Lab5, which is some advanced challenges! 

<b><font color="red" size=3>State Your Contribution (Todo)</font></b>

Before submitting, please kindly fill the Microsoft forms (released on Moodle later) that claim contributions of your teammates. You need to specify your roles in your team, such as coding for each part, debugging, or looking up materials. 

Note that the form you submitted should get consent from ALL of your team members. TA will confirm that with all of you afterwards.

<b><font color="red" size=3>About your report and presentation</font></b>

You are required to submmit a report (no more than 2 pages) as well as a team presentation. Multiple groups would choose this lab and how to stand out? Here are some hints:

- You can write about your observations and your findings. Or you may change some parameters to see what is changed. That is distinct and show your reflections
- If you intend to finish Lab 5, you can emphasis your report and pre on what you have changed to make our algorithm more robust and accurate;
- You are also encouraged to refer to some papers listed in Lab 5 and make a summary of current techniques

The key is, you need to tell us something different. I hope you can benifit from our course.